In [1]:
import numpy as np
import math
import pandas as pd

In [2]:
'''
定义辅助函数：
函数功能：计算sigmoid函数值
参数说明：
    inX：数值型数据
返回：
    s：经过sigmoid函数计算后的函数值
'''
def sigmoid(inX):
    s = 1/(1+ np.exp(-inX))
    return s

In [3]:
'''
为什么要标准化，见（pdf）
标准化函数
函数功能：标准化（期望为0，方差为1）
参数说明：
    xMat：特征矩阵
返回：
    inMat: 标准化之后的特征函数
'''
def regularize(xMat):
    inMat = xMat.copy()
    inMeans = np.mean(inMat,axis = 0) # axix = 0 ,压缩行，对各列求均值，返回1*n的矩阵 
    #print(inMeans)---->[[0.0300122 6.57611  ]]
    inVar = np.std(inMat,axis = 0) #求每一列的标准差
    #print(inVar)---->[[1.16447043 4.6216594 ]]
    inMat = (inMat - inMeans)/inVar # 标准化
    return inMat

In [4]:
'''
函数功能： 使用BGD求解逻辑回归

使用的梯度下降法的矩阵方式（见pdf）

参数说明：
    dataSet ： DF数据集
    alpha ： 步长
    maxCycle：最大迭代次数
返回：
    weights： 各特征权重值
'''
def BGD_LR(dataSet,alpha = 0.001,maxCycles = 500):
    xMat = np.mat(dataSet.iloc[:,:-1].values) # 提取出特征值
    yMat = np.mat(dataSet.iloc[:,-1].values).T # 提取出标签的值并转置
    xMat = regularize(xMat) # 标准化
    m , n = xMat.shape # 行，列:100,2
    weights = np.zeros((n,1)) # 生成一个n*1的列表
    # print(weights)---->[[0.]
    #                     [0.]
    #                     ]
    for i in range(maxCycles): # 迭代maxCycles次
        grad = xMat.T * (xMat * weights - yMat) / m # 各个特征的梯度
        weights = weights - alpha * grad # 更新权重值
    return weights

In [5]:

'''
函数功能： 使用SGD求解逻辑回归

使用的梯度下降法的矩阵方式（见pdf）

参数说明：
    dataSet ： DF数据集
    alpha ： 步长
    maxCycle：最大迭代次数
返回：
    weights： 各特征权重值
'''
def SGD_LR(dataSet,alpha = 0.001,maxCycles = 500):
    dataSet = dataSet.sample(maxCycles,replace = True)
    # maxCycles的含义是抽样的个数，是整数;smaple()函数中的replace参数的意思是：是否允许抽样值重复
    dataSet.index = range(dataSet.shape[0]) # 重新编排索引值
    xMat = np.mat(dataSet.iloc[:,:-1].values) # 提取出特征值
    yMat = np.mat(dataSet.iloc[:,-1].values).T # 提取出标签的值并转置
    xMat = regularize(xMat) # 标准化
    m , n = xMat.shape # 行，列:100,2
    weights = np.zeros((n,1)) # 生成一个n*1的列表
    # print(weights)---->[[0.]
    #                     [0.]
    #                     ]
    for i in range(m): # 迭代m次
        grad = xMat.T * (xMat * weights - yMat) / m # 各个特征的梯度
        weights = weights - alpha * grad # 更新权重值
    return weights

In [9]:
train = pd.read_table('horseColicTraining.txt',header=None)
train.head()
train.shape
#print(train.shape)----->(299,22)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 299 entries, 0 to 298
Data columns (total 22 columns):
0     299 non-null float64
1     299 non-null float64
2     299 non-null float64
3     299 non-null float64
4     299 non-null float64
5     299 non-null float64
6     299 non-null float64
7     299 non-null float64
8     299 non-null float64
9     299 non-null float64
10    299 non-null float64
11    299 non-null float64
12    299 non-null float64
13    299 non-null float64
14    299 non-null float64
15    299 non-null float64
16    299 non-null float64
17    299 non-null float64
18    299 non-null float64
19    299 non-null float64
20    299 non-null float64
21    299 non-null float64
dtypes: float64(22)
memory usage: 51.5 KB


In [16]:
test = pd.read_table('horseColicTest.txt',header=None)
test.head()
test.shape
#print(train.shape)----->(67,22)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67 entries, 0 to 66
Data columns (total 22 columns):
0     67 non-null int64
1     67 non-null int64
2     67 non-null float64
3     67 non-null int64
4     67 non-null int64
5     67 non-null int64
6     67 non-null int64
7     67 non-null int64
8     67 non-null int64
9     67 non-null int64
10    67 non-null int64
11    67 non-null int64
12    67 non-null int64
13    67 non-null int64
14    67 non-null float64
15    67 non-null int64
16    67 non-null int64
17    67 non-null float64
18    67 non-null float64
19    67 non-null int64
20    67 non-null float64
21    67 non-null int64
dtypes: float64(5), int64(17)
memory usage: 11.6 KB


从上面测试集和训练集的info()中可以看出他们没有确实的数据

In [27]:
'''
函数功能: 给定测试数据和权重，返回类别标签
参数说明:
    inX : 测试数据
    weights: 特征权重
返回：
    标签
'''
def classify(inX,weights):
    p = sigmoid(inX * weights)
    if p < 0.5:
        return 0
    else:
        return 1

In [32]:
'''
函数功能：logistic分类模型 
参数说明：    
    train：测试集    
    test：训练集    
    method: 训练方法
    alpha：步长    
    maxCycles：大迭代次数 
返回：    
    retest:预测好标签的测试集
'''
def get_acc(train , test ,method, alpha = 0.001 , maxCycles = 5000):
    weights = method(train,alpha=alpha , maxCycles=maxCycles)
    xMat = np.mat(test.iloc[:,:-1].values)
    xMat = regularize(xMat)
    result = []
    for inX in xMat:
        label = classify(inX,weights)
        result.append(label)
    retest = test.copy()
    retest['predict'] = result
    acc = (retest.iloc[:,-1] == retest.iloc[:,-2]).mean()
    #print(f'模型准确率：{acc}')
    return acc
    #return retest
        

In [35]:
if __name__ == '__main__':
    test = pd.read_table('horseColicTest.txt',header=None)
    train = pd.read_table('horseColicTraining.txt',header=None)
    for i in range(20):
        print(f"第{i+1}次训练：")
        print("使用BGD方法：准确率{%.6f}"%get_acc(train,test,BGD_LR,alpha=0.001,maxCycles=5000))
        print("使用SGD方法：准确率{%.6f}"%get_acc(train,test,SGD_LR,alpha=0.001,maxCycles=5000))

        

第1次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第2次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.716418}
第3次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第4次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.701493}
第5次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第6次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第7次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第8次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第9次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.716418}
第10次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第11次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.716418}
第12次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第13次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第14次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.686567}
第15次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第16次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第17次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.746269}
第18次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.716418}
第19次训练：
使用BGD方法：准确率{0.716418}
使用SGD方法：准确率{0.731343}
第20次训练：
使用BGD方法：准确率{0